In [ ]:
import os
from glob import glob
import yaml
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
#bing_image_search api를 활용하여 각 차종/브랜드 별 차량 이미지 수집
!python bing_search_api.py --query "Keywords" --output 원본 데이터 경로

In [ ]:
#yolov5 pre-trained 모델을 활용하여 수집된 이미지 중 차량을 포함한 이미지만 차량이 있는 부분을 마커표시, 라벨링하여 텍스트 파일로 저장
# yolo detect.py 파일 일부 수정
!python detect.py --source 원본 데이터 경로 --weights yolov5s.pt --conf 0.25

In [ ]:
#이미지에 차량이 포함되지 않아 이미지 파일이 생성되지 않은 이미지 파일 제거
txt_list = glob('/원본 데이터 경로/*.txt')
img_list = glob('/원본 데이터 경로/*')

txt_name_list = list(map(lambda i: i[i.rfind("/") + 1 : -4], txt_list)
img_name_list = list(map(lambda i: i[i.rfind("/") + 1 : -4], img_list)

for idx, img_file in enumerate(img_list):
    if img_file not in txt_list:
        os.remove(img_list[idx])

# train/test data 분리
data_img_list = glob('/원본 데이터 경로/*.png') + glob('/원본 데이터 경로/*.jpg')

train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state = 2000)

#이미지 데이터 경로 텍스트 파일로 저장
with open('/데이터 경로/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/데이터 경로/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

#yaml 파일 수정
with open('/데이터 경로/mydata.yaml','r') as f:
    data = yaml.load(f)

data['train'] = '/데이터 경로/train.txt'
data['val'] = '/데이터 경로/val.txt'
data['names'] : [라벨 목록]

with open('/content/drive/MyDrive/yolov5/dataset/data.yaml','w') as f:
    yaml.dump(data, f)

In [ ]:
!python train.py --img 512 --batch 16 --epoch 100 --data /데이터 경로/mydata.yaml --name 모델명

In [ ]:
#stylegan2-ada로 생성한 차량 디자인 이미지 파일을 classification 하여 텍스트 파일(car_kinds.txt, car_brands.txt)에 파일명, 차종, 브랜드를 저장
#yolo detect.py 파일 일부 수정
!python detect.py --source 생성한 이미지 데이터 경로 --weights 차종분류모델.pt --conf 0.25
!python detect.py --source 생성한 이미지 데이터 경로 --weights 브랜드분류모델.pt --conf 0.25

In [ ]:
#분류된 이미지파일을 폴더별로 이동
brands = {}

f_brand = open("/데이터 경로/car_brands.txt", 'r')

name_brands = f_brand.readlines()

for name_brand in name_brands:
    file_name, brands = line.split(",")
    brands.update({file_name:brands})

f_brand.close()

f_kind = open("/데이터 경로/car_kinds.txt", 'r')

name_kinds = f_kind.readlines()

for name_kind in name_kinds:
    file_name, kinds = line.split(",")
    shutil.move("/저장경로/" + file_name, "/저장경로/"+ kinds + "/" + brands[file_name] + "/" + file_name)

f_kind.close()